# Merging datasets

In [273]:
import pandas as pd

In [274]:
path_mercado = 'C:/workspace/Desafio Grão Direto IA/Data/mercado-desafio-tratado.xlsx'
path_transacoes = 'C:/workspace/Desafio Grão Direto IA/Data/transações-desafio-tratado.xlsx'

mercado = pd.read_excel(path_mercado)
transacoes =  pd.read_excel(path_transacoes)

In [275]:
seller_transactioned_sum = transacoes.groupby('Seller ID')['Transactioned'].sum()
seller_ids = seller_transactioned_sum[seller_transactioned_sum > 2].index
transacoes = transacoes[transacoes['Seller ID'].isin(seller_ids)]

In [276]:
model_df = pd.merge(transacoes,mercado,on='Date',how='left')

In [ ]:
model_df

# Separando o dataset com temporal_train_test

In [278]:
# from sktime.forecasting.model_selection import temporal_train_test_split

In [279]:
# seller_id = 100000060
# time_serie = model_df[model_df['Seller ID'] == seller_id]
# time_serie.drop(columns=['Seller ID','Day of Week','Last Week','Last Month'],inplace=True)
# time_serie['Change'] = time_serie['Change'].astype('Int64')
# time_serie

# Lag de um dia na nossa time serie

In [280]:
# time_serie.set_index('Date',inplace=True)

In [281]:
# target = time_serie.pop('Transactioned')
# y = pd.DataFrame(data=target,columns=['Transactioned'])
# y

In [282]:
# time_serie = time_serie.shift(1)
# time_serie.drop(pd.to_datetime('2024-01-30'), inplace=True)
# time_serie

In [283]:
# date_to_pred = pd.to_datetime('2024-11-04')
# time_serie = time_serie[time_serie.index < date_to_pred]
# y = y[y.index<date_to_pred]
# y.drop(pd.to_datetime('2024-01-30'), inplace=True)
# y

In [284]:
# time_serie

In [285]:
# y_train, y_test, X_train, X_test = temporal_train_test_split(y,time_serie,test_size=20)

# Creating forecasting

In [286]:
# from sktime.forecasting.base import ForecastingHorizon
# import numpy as np

# fh = ForecastingHorizon(np.arange(1,len(y_test)+2),is_relative=True)

In [287]:
# from xgboost import XGBRegressor

# # Calculando scale_pos_weight
# n_pos = len(y_train[y_train['Transactioned'] == 1])
# n_neg = len(y_train[y_train['Transactioned'] == 0])
# scale_pos_weight = n_neg / n_pos

# xgb_model = XGBRegressor(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     scale_pos_weight=scale_pos_weight
# )

In [288]:
# from sktime.forecasting.compose import make_reduction

# forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

In [289]:
# X_train.info()

In [290]:
# forecaster.fit(y=y_train,X=X_train)

In [291]:
# y_pred = forecaster.predict(fh=fh,X=X_test)
# y_pred

In [292]:
# from sktime.utils.plotting import plot_series
# plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])

In [293]:
# # Análise de erros
# errors = abs(y_test['Transactioned'].values - y_pred.values)
# print("Erro Médio Absoluto (MAE):", errors.mean())

# Iterando em sellers

In [294]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from xgboost import XGBRegressor
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## PCA

In [ ]:
# def pca(df):
#     # Define the features variable
#     features = df.columns.tolist()

#     # 1. Normalização das features
#     scaler = StandardScaler()
#     df_scaled = scaler.fit_transform(df)

#     # 2. Aplicar PCA
#     pca = PCA()
#     pca.fit(df_scaled)

#     # 3. Variância explicada
#     explained_variance_ratio = pca.explained_variance_ratio_
#     cumulative_variance = np.cumsum(explained_variance_ratio)

#     # 4. Determinar o número ideal de componentes para atingir, por exemplo, 80% da variância
#     threshold = 0.80
#     num_components = np.argmax(cumulative_variance >= threshold) + 1

#     print(f'Número de componentes para atingir {threshold*100:.0f}% da variância: {num_components}')

#     # 5. Visualizar variância explicada
#     plt.figure(figsize=(12, 7))
#     plt.bar(range(1, len(explained_variance_ratio)+1), explained_variance_ratio, alpha=0.6, color='g', 
#             label='Individual Explained Variance')
#     plt.plot(range(1, len(cumulative_variance)+1), cumulative_variance, marker='o', linestyle='-', color='r', 
#             label='Cumulative Explained Variance')
#     plt.axhline(y=threshold, color='b', linestyle='--', label=f'{threshold*100:.0f}% Variance Threshold')
#     plt.axvline(x=num_components, color='purple', linestyle='--', label=f'{num_components} Components')
#     plt.xlabel('Principal Components')
#     plt.ylabel('Explained Variance')
#     plt.title('Explained Variance by Principal Components')
#     plt.xticks(range(1, len(explained_variance_ratio)+1))
#     plt.legend(loc='upper left')
#     plt.grid(True)
#     plt.show()
#     loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(len(explained_variance_ratio))],
#                             index=df.columns)
#     loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(len(explained_variance_ratio))],
#                             index=features)

#     # Exibir os loadings mais importantes para os principais componentes
#     print("Loadings (Pesos das Features nos Componentes Principais):")
#     print(loadings)

#     # Soma absoluta dos loadings para os 4 primeiros PCs
#     top_n_pcs = 4  # Quantos PCs você quer considerar
#     sum_abs_loadings = loadings.iloc[:, :top_n_pcs].abs().sum(axis=1)

#     # Adiciona a soma como uma nova coluna no DataFrame original das features
#     loadings['Sum_Absolute_Loadings_Top4'] = sum_abs_loadings

#     # Ordena as features por importância (soma decrescente)
#     sorted_features = loadings['Sum_Absolute_Loadings_Top4'].sort_values(ascending=False)

#     # Exibir as features mais importantes
#     print("Soma Absoluta dos Pesos nos 4 Primeiros PCs (por Feature):")
#     print(sorted_features)

#     # Identificar as features mais importantes para os primeiros componentes principais
#     top_features = {}
#     for i in range(num_components):
#         pc_loadings = loadings[f'PC{i+1}'].abs().sort_values(ascending=False)
#         top_features[f'PC{i+1}'] = pc_loadings.head(3).index.tolist()

#     print("\nFeatures mais importantes para cada componente principal:")
#     for pc, features in top_features.items():
#         print(f'{pc}: {features}')

# pca_df = model_df[model_df['Seller ID'] == 100000060]
# pca_df.set_index('Date',inplace=True)
# pca_df = pca_df.drop(columns=['Transactioned','Seller ID',])

# pca(pca_df)

## Modelo

In [309]:
def process_seller(seller_id):
    print(f"Processando Seller ID: {seller_id}")
    
    # Filtrar os dados para o Seller ID atual
    time_serie = model_df[model_df['Seller ID'] == seller_id]
    time_serie = time_serie.drop(columns=['Seller ID','Close','CBOT','Last Week','Last Month','Last Transaction','Price','Change','Percentage Change'])
    
    # Preparação da série temporal
    time_serie.set_index('Date', inplace=True)
    target = time_serie.pop('Transactioned')
    y = pd.DataFrame(data=target, columns=['Transactioned'])
    time_serie = time_serie.shift(1)
    
    # Remover valores fora do intervalo de datas desejado
    date_to_pred = pd.to_datetime('2024-11-04')
    first_date = pd.to_datetime('2024-01-30')
    mask = (time_serie.index > first_date) & (time_serie.index < date_to_pred)
    time_serie = time_serie.loc[mask]
    y = y.loc[mask]

    # Dividir os dados em treino e teste
    y_train, y_test, X_train, X_test = temporal_train_test_split(y, time_serie, test_size=20)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    # Configuração do Forecasting Horizon
    fh = ForecastingHorizon(pd.DatetimeIndex([date_to_pred]), is_relative=False)
    
    # Calculando scale_pos_weight
    n_pos = len(y_train[y_train['Transactioned'] == 1])
    n_neg = len(y_train[y_train['Transactioned'] == 0])
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
    # Configurar o XGBRegressor
    xgb_model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        scale_pos_weight=scale_pos_weight
    )
    
    # Criar e treinar o modelo
    forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")
    forecaster.fit(y=y_train, X=X_train)
    
    # Previsão
    y_pred = forecaster.predict(fh=fh, X=X_test)
    # y_pred_04 = y_pred.loc['2024-11-04']
    
    # Análise de erros
    errors = abs(y_test['Transactioned'].values - y_pred.values)
    mae = errors.mean()
    print(f"Erro Médio Absoluto (MAE) para Seller ID {seller_id}: {mae}")
    
    # # Plot das séries
    # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    # plt.title(f"Previsão para Seller ID: {seller_id}")
    # plt.show()
    
    # Armazenar os resultados
    return{
        "Seller ID": seller_id,
        "Date": y_pred.index.values[0],  # Extract the single date value
        "Score": y_pred['Transactioned'].values[0],  # Extract the single score value
        "MAE": mae
    }

In [ ]:
# Lista de Seller IDs
seller_ids = model_df['Seller ID'].unique()

# Resultados para armazenar os erros por vendedor
results = []

with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_seller,seller_ids))

# Criar DataFrame com os resultados
results_df = pd.DataFrame(results)
results_df

In [ ]:
media = results_df['MAE'].mean()
media

In [312]:
users_that_made_transaction_on_target_date = [
    100000060,
    100000141,
    100000192,
    100000310,
    100000348,
    100001098,
    100001362,
    100001525,
    100002035,
    100002271,
    100002332,
    100002339,
    100002402,
    100002426,
]

In [ ]:
first_200_predictions = results_df.sort_values(
    ascending=False, by="Score"
).head(200)


first_200_predictions[
    first_200_predictions["Seller ID"].isin(users_that_made_transaction_on_target_date)
]

# Coisas a fazer
- PCA
- GRIDSEARCH
- CONCURRENT FUTURES V